In [1]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [2]:
import os

import keras
import keras_nlp
import tensorflow as tf
import pandas as pd
from datasets import load_dataset
import tensorflow_datasets as tfds

2024-05-27 14:40:21.022026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 14:40:21.022146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 14:40:21.142414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


# Load Dataset

In [15]:
dataset = load_dataset("nyu-mll/glue", "stsb")
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

In [16]:
train_data, validation_data, test_data = dataset['train'], dataset['validation'], dataset['test']

In [17]:
train_df = train_data.to_pandas()
validation_df = validation_data.to_pandas()
test_df = test_data.to_pandas()

In [18]:
train_df.head()

,sentence1,sentence2,label,idx
0,A plane is taking off.,An air plane is taking off.,5.00,0
1,A man is playing a large flute.,A man is playing a flute.,3.80,1
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80,2
3,Three men are playing chess.,Two men are playing chess.,2.60,3
4,A man is playing the cello.,A man seated is playing the cello.,4.25,4


In [19]:
train_df = train_df.drop('idx', axis=1)
validation_df = validation_df.drop('idx', axis=1)
test_df = test_df.drop('idx', axis=1)

In [20]:
X_train, y_train = train_df.drop('label', axis=1), train_df.label
X_validation, y_validation = validation_df.drop('label', axis=1), validation_df.label
X_test, y_test = test_df.drop('label', axis=1), test_df.label

In [24]:
train_tf = tf.data.Dataset.from_tensor_slices((X_train['sentence1'], X_train['sentence2'], y_train.values))
validation_tf = tf.data.Dataset.from_tensor_slices((X_validation['sentence1'], X_validation['sentence2'], y_validation.values))
test_tf = tf.data.Dataset.from_tensor_slices((X_test['sentence1'], X_test['sentence2'], y_test.values))

In [25]:
#ConvertMapDataset
train_tf = train_tf.map(lambda sentence1, sentence2, y : {'sentence1': sentence1, 'sentence2':sentence2, 'label':y})
validation_tf = validation_tf.map(lambda sentence1, sentence2, y : {'sentence1': sentence1, 'sentence2':sentence2, 'label':y})
test_tf = test_tf.map(lambda sentence1, sentence2, y : {'sentence1': sentence1, 'sentence2':sentence2, 'label':y})

In [28]:
#ConvertPrefetcDataset
train_data = train_tf.prefetch(1)
validation_data = validation_tf.prefetch(1)
test_data = test_tf.prefetch(1)

# Parameter Dataset

In [35]:
TRAIN_BATCH_SIZE = 6
VALIDATION_BATCH_SIZE = 8

TRAIN_NUM_BATCHES = 300
VALIDATION_NUM_BATCHES = 40

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Preprocessing Data

In [36]:
def change_range(x):
    return (x/2.5)-1

In [39]:
def preprocessing_data(dataset, num_batches, batch_size):
    dataset = dataset.map(
        lambda z: (
            [z['sentence1'], z['sentence2']],
            [tf.cast(change_range(z['label']), tf.float32)]
        ),
        num_parallel_calls = AUTOTUNE,
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.take(num_batches)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [40]:
train_data = preprocessing_data(train_data, TRAIN_NUM_BATCHES, TRAIN_BATCH_SIZE)
validation_data = preprocessing_data(validation_data, VALIDATION_NUM_BATCHES, VALIDATION_BATCH_SIZE)

In [43]:
for x, y in train_data:
    for i, example in enumerate(x):
        print(f"sentence1 : {example[0]}")
        print(f"sentence2 : {example[1]}")
        print(f"similarity : {y[i]}")
        break

sentence1 : b'A plane is taking off.'
sentence2 : b'An air plane is taking off.'
similarity : [1.]
sentence1 : b'A man is smoking.'
sentence2 : b'A man is skating.'
similarity : [-0.8]
sentence1 : b'A man is playing a flute.'
sentence2 : b'A man is playing a bamboo flute.'
similarity : [0.5468]
sentence1 : b"A cat is rubbing against baby's face."
sentence2 : b'A cat is rubbing against a baby.'
similarity : [0.52]
sentence1 : b'A woman peels a potato.'
sentence2 : b'A woman is peeling a potato.'
similarity : [1.]
sentence1 : b'A man is riding a mechanical bull.'
sentence2 : b'A man rode a mechanical bull.'
similarity : [0.6]
sentence1 : b'A dog rides a skateboard.'
sentence2 : b'A dog is riding a skateboard.'
similarity : [1.]
sentence1 : b'A baby tiger is playing with a ball.'
sentence2 : b'A baby is playing with a doll.'
similarity : [-0.36]
sentence1 : b'A kid is playing guitar.'
sentence2 : b'A boy is playing a guitar.'
similarity : [0.52]
sentence1 : b'A woman is slicing an onion.'